In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re

In [2]:
reviews_train = []
for line in open('dataset/full_test.txt', 'r'):
    
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('dataset/full_train.txt', 'r'):
    
    reviews_test.append(line.strip())

In [3]:
reviews_train[5]

'I saw this film on September 1st, 2005 in Indianapolis. I am one of the judges for the Heartland Film Festival that screens films for their Truly Moving Picture Award. A Truly Moving Picture "...explores the human journey by artistically expressing hope and respect for the positive values of life." Heartland gave that award to this film.<br /><br />This is a story of golf in the early part of the 20th century. At that time, it was the game of upper class and rich "gentlemen", and working people could only participate by being caddies at country clubs. With this backdrop, this based-on-a-true-story unfolds with a young, working class boy who takes on the golf establishment and the greatest golfer in the world, Harry Vardon.<br /><br />And the story is inspirational. Against all odds, Francis Ouimet (played by Shia LaBeouf of "Holes") gets to compete against the greatest golfers of the U.S. and Great Britain at the 1913 U.S. Open. Francis is ill-prepared, and has a child for a caddy. (T

In [4]:
import re


#REPLACE_NO_SPACE = re.compile("virginamerica")
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\@)|(\d+)|(\virginamerica)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [5]:
reviews_train_clean[5]

'i saw this film on september st  in indianapolis i am one of the judges for the heartland film festival that screens films for their truly moving picture award a truly moving picture explores the human journey by artistically expressing hope and respect for the positive values of life heartland gave that award to this film this is a story of golf in the early part of the th century at that time it was the game of upper class and rich gentlemen and working people could only participate by being caddies at country clubs with this backdrop this based on a true story unfolds with a young working class boy who takes on the golf establishment and the greatest golfer in the world harry vardon and the story is inspirational against all odds francis ouimet played by shia labeouf of holes gets to compete against the greatest golfers of the us and great britain at the  us open francis is ill prepared and has a child for a caddy the caddy is hilarious and motivational and steals every scene he ap

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 12500 else 0 for i in range(25000)]

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
#     Accuracy for C=0.01: 0.87616
#     Accuracy for C=0.05: 0.88736
#     Accuracy for C=0.25: 0.88768
#     Accuracy for C=0.5: 0.88336
#     Accuracy for C=1: 0.8784

/home/poochi/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Accuracy for C=0.01: 0.87616
Accuracy for C=0.05: 0.88736
Accuracy for C=0.25: 0.88768
Accuracy for C=0.5: 0.88336
Accuracy for C=1: 0.8784


In [8]:
final_model = LogisticRegression(C=0.054)
final_model.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_model.predict(X_test)))
# Final Accuracy: 0.88048

Final Accuracy: 0.88048


In [16]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
    
print("___________________________________")
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)
    
#     ('worst', -1.367978497228895)
#     ('waste', -1.1684451288279047)
#     ('awful', -1.0277001734353677)
#     ('poorly', -0.8748317895742782)
#     ('boring', -0.8587249740682945)

('excellent', 0.9185348458306545)
('hilarious', 0.806108452299701)
('amazing', 0.7861385201487513)
('perfect', 0.7203762355164427)
('great', 0.7202248540331433)
___________________________________
('worst', -1.4505203454128626)
('waste', -1.27052217836691)
('awful', -1.1005383605309653)
('terrible', -0.9962672087548639)
('boring', -0.9103011437126847)
